In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
N_ITER = 10

def get_importances_cutoff(feature_names, X, y):
    reg = RandomForestRegressor(n_estimators=50, n_jobs=8)

    importances = np.zeros(X.shape[1]*2)
    for _ in xrange(N_ITER):
        Xc = []
        for col in X.T:
            Xc.append(np.random.choice(col, size=col.shape[0]))
        Xc = np.array(Xc).T
        Xn = np.hstack((X, Xc))
        reg.fit(Xn, y)
        importances += np.array(reg.feature_importances_)
        
    importances /= N_ITER
    indices = np.argsort(-importances)

    m = np.max(importances[indices[indices >= X.shape[1]]])
    sd = np.std(importances[indices[indices >= X.shape[1]]])
    diffs = importances[indices][:-1] - importances[indices][1:]
    cutoff = importances[indices][np.where(diffs > 2*sd)[0][-1]]

    indices = indices[indices < X.shape[1]]
    
    return feature_names[indices], importances[indices], cutoff


def feature_ranking(fname, use_cutoff=True):

    def column_filter(col):
        return "ifreq" not in col

    data = pd.read_csv(fname, sep="\t")
    data = data[["chr", "begin", "end"]+[col for col in data.columns[3:] if column_filter(col)]]
    data = data.fillna(0)
    
    y = (data["end"]-data["begin"]).values
    X = data.iloc[:, 4:]
    
    names, r, cutoff = get_importances_cutoff(X.columns, X.values, y)
    
    if not use_cutoff:
        cutoff = 0
    
    r1 = r[r > cutoff]
    r2 = r[r <= cutoff]
    names = names[r > cutoff]
    
    fig, ax = plt.subplots()
    
    ax.bar(np.arange(r1.shape[0]), r1, color="blue")
    ax.bar(r1.shape[0]+np.arange(r2.shape[0]), r2, color="red")
    
    return pd.DataFrame({"RI": r1, "feature": names})

# Feature rankings

Feature rankings contain important features as ranked by Random Forest regression. Cutoff was chosen based on so called contrast attributes - random permutations of the original attributes. Standard deviation (SD) of importance scores for contrast attributes was calculated. The last attribute in the ranking with importance larger then the importance of subsequent attribute by at least 2 SD constituted a cutoff.

Plots present scores of attributes considered important (blue) and the rest of attributes falling below cutoff (red).

## H3K4me3 K562

In [ ]:
feature_ranking("../data/processed_peaks/K562/K562_H3K4me3_mods.csv")

## H3K4me3 MCF7

No cutoff was used because of small number of attributes.

In [ ]:
feature_ranking("../data/processed_peaks/MCF7/MCF7_H3K4me3_mods.csv", False)

## H3K27ac K562

In [ ]:
feature_ranking("../data/processed_peaks/K562/K562_H3K27ac_mods.csv")

## H3K27ac MCF7

No cutoff was used because of small number of attributes.

In [ ]:
feature_ranking("../data/processed_peaks/MCF7/MCF7_H3K27ac_mods.csv", False)